<a href="https://colab.research.google.com/github/timroelofs123/face_reaging/blob/main/notebooks/gradio_demos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Re-Aging Demos with Gradio
To try out the Gradio Demos using Google Colab, run the cells below.
Be sure to select a runtime with GPU (e.g. `Runtime > Change Runtime Type > T4 (GPU)`) to make the model run fast.

You can choose to either run the regular inference demo or the video demo. The demo will open in a new tab.

## Downloading files and model

In [ ]:
!git clone https://github.com/timroelofs123/face_reaging

In [ ]:
!git clone https://huggingface.co/timroelofs123/face_re-aging

In [ ]:
%mv face_re-aging/best_unet_model.pth face_reaging/
%cd face_reaging/

## Installing requirements

In [ ]:
%pip install gradio face_recognition antialiased_cnns kaleido --quiet

## Running demo

Re-age your photo or video with the three demos.

In [ ]:
!python scripts/gradio_demo.py --model_path 'best_unet_model.pth'